In [ ]:
!pip install "unsloth[kaggle] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
!pip install "git+https://github.com/huggingface/transformers.git"

In [ ]:
!pip install trl

In [ ]:
!pip install bitsandbytes
from unsloth import FastLanguageModel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from trl import SFTTrainer
from transformers import TrainingArguments

In [ ]:
from datasets import load_dataset

In [ ]:
max_seq_length = 4096

In [ ]:
dataset = load_dataset("imdb",split="train")

In [ ]:
dataset

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype= None,
    load_in_4bit=True
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
        "dense_h_to_4h", "dense_4h_to_h"
    ],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length
)

In [ ]:
trainer = SFTTrainer(
    model = model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=60,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        output_dir="unsloth-test",
        optim="adamw_8bit",
        seed=3407
    ),
)
trainer.train()

In [ ]:
inputs = tokenizer(
    [
        "I really like the movie because it shows emotions and talks humanity"
    ],
    return_tensors="pt",
).to("cuda")

In [ ]:
inputs

In [ ]:
# Generate output from model
outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.2,
    eos_token_id=tokenizer.eos_token_id,
)

# Decode and clean the output
decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
cleaned = [text.replace("\n", " ").strip() for text in decoded]

# Print the cleaned result
print(cleaned[0])

In [ ]:
outputs

In [ ]:
tokenizer.batch_decode(outputs)

In [ ]:
model.save_pretrained("lora_model")

In [ ]:
!ls -R /content

In [ ]:
# Zip all required folders
!zip -r /content/all_colab_folders.zip \
  /content/huggingface_tokenizers_cache \
  /content/lora_model \
  /content/sample_data \
  /content/unsloth-test \
  /content/unsloth_compiled_cache \
  /content/wandb

# Download the zip file
from google.colab import files
files.download('/content/all_colab_folders.zip')